In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import duckdb
import polars as pl
import sys
sys.path.append('../') 
import upsert_utils

In [3]:
duckdb.sql("""
attach '../../../db.sqlite3';
use db;
""")

In [ ]:
duckdb.sql("""
rollback;
""")

In [4]:
duckdb.sql("""
begin transaction;
""")

In [5]:
all_columns = duckdb.sql("""
select * from read_csv('ProdModule.csv');
""").pl()

In [6]:
ob_dimensions = duckdb.sql(r"""
select
    csv_row_id,
    columns('^Dimension__(.+)') as '\1',
from all_columns
""")
ob_dimensions

┌────────────┬────────────┬─────────────┬─────────────┬──────────────┐
│ csv_row_id │ Width_Unit │ Width_Value │ Height_Unit │ Height_Value │
│   int64    │  varchar   │   double    │   varchar   │    double    │
├────────────┼────────────┼─────────────┼─────────────┼──────────────┤
│          1 │ m          │       0.992 │ m           │         1.64 │
│          2 │ m          │       0.992 │ m           │         1.64 │
│          3 │ m          │       0.992 │ m           │         1.64 │
│          4 │ m          │       0.992 │ m           │         1.64 │
│          5 │ m          │       0.992 │ m           │         1.64 │
│          6 │ m          │       0.995 │ m           │        1.646 │
│          7 │ m          │       0.995 │ m           │        1.646 │
│          8 │ m          │       0.995 │ m           │        1.646 │
│          9 │ m          │       0.995 │ m           │        1.646 │
│         10 │ m          │       0.995 │ m           │        1.646 │
│     

In [7]:
ob_prod_cells = duckdb.sql(r"""
select
    csv_row_id,
    columns('^ProdCell__(.+)') as '\1',
from all_columns
""")
ob_prod_cells

┌────────────┬──────────────────────────┐
│ csv_row_id │ CellTechnologyType_Value │
│   int64    │         varchar          │
├────────────┼──────────────────────────┤
│          1 │ MonoSi                   │
│          2 │ MonoSi                   │
│          3 │ MonoSi                   │
│          4 │ MonoSi                   │
│          5 │ MonoSi                   │
│          6 │ PolySi                   │
│          7 │ PolySi                   │
│          8 │ PolySi                   │
│          9 │ MonoSi                   │
│         10 │ PolySi                   │
│          · │   ·                      │
│          · │   ·                      │
│          · │   ·                      │
│       9991 │ MonoSi                   │
│       9992 │ MonoSi                   │
│       9993 │ MonoSi                   │
│       9994 │ MonoSi                   │
│       9995 │ MonoSi                   │
│       9996 │ MonoSi                   │
│       9997 │ MonoSi             

In [8]:
ob_prod_modules = all_columns.drop('^.+__.+$')
duckdb.sql("""
describe ob_prod_modules;
""")

┌─────────────────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│                   column_name                   │ column_type │  null   │   key   │ default │  extra  │
│                     varchar                     │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ csv_row_id                                      │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdMfr_Value                                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdCode_Value                                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description_Value                               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ModuleArea_Unit                                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ModuleArea_Value                            

In [9]:
ob_prod_module_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        csv_row_id,
        ModuleArea_Unit,
        ModuleArea_Value,
        CellsInSeries_Value,
        CellStringsParallelQuantity_Value,
        IsBIPV_Value,
        TemperatureNOCT_Unit,
        TemperatureNOCT_Value,
        TemperatureCoefficientMaximumPower_Unit,
        TemperatureCoefficientMaximumPower_Value,
        TemperatureCoefficientShortCircuitCurrent_Unit,
        TemperatureCoefficientShortCircuitCurrent_Value,
        TemperatureCoefficientOpenCircuitVoltage_Unit,
        TemperatureCoefficientOpenCircuitVoltage_Value,
        TemperatureCoefficientMaxPowerCurrent_Unit,
        TemperatureCoefficientMaxPowerCurrent_Value,
        TemperatureCoefficientMaxPowerVoltage_Unit,
        TemperatureCoefficientMaxPowerVoltage_Value,
    from ob_prod_modules
    """),
    target_table_name='server_prodmodule',
    pk='product_ptr_id',
    fk_fills=[
        ('product_ptr_id', dict(
            rows=duckdb.sql("""
            select
                csv_row_id,
                replace(cast(uuidv4() as varchar), '-', '') as ProdID_Value,
                ProdMfr_Value,
                ProdCode_Value,
                coalesce(Description_Value, '') as Description_Value,
                'ProdModule' as ProdType_Value,
            from ob_prod_modules
            """),
            target_table_name='server_product',
            pk='id',
            superclass=True,
            fk_fills=[
                ('Dimension_id', dict(rows=ob_dimensions, target_table_name='server_dimension', pk='id')),
            ]
        )),
        ('ProdCell_id', dict(
            rows=ob_prod_cells,
            target_table_name='server_prodcell',
            pk='product_ptr_id',
            fk_fills=[
                ('product_ptr_id', dict(
                    rows=duckdb.sql("""
                    select
                        csv_row_id,
                        replace(cast(uuidv4() as varchar), '-', '') as ProdID_Value,
                        null as ProdCode_Value,
                        'ProdCell' as ProdType_Value,
                    from ob_prod_cells
                    """),
                    target_table_name='server_product',
                    pk='id',
                    superclass=True,
                    fk_fills=[
                        ('Dimension_id', dict(target_table_name='server_dimension', pk='id')),
                    ]
                )),
            ]
        )),
        ('ProdGlazing_id', dict(target_table_name='server_prodglazing', pk='id')),
    ]
)
ob_prod_module_ids

┌────────────┬────────────────┐
│ csv_row_id │ product_ptr_id │
│   int64    │     int64      │
├────────────┼────────────────┤
│          1 │            864 │
│          2 │            865 │
│          3 │            866 │
│          4 │            867 │
│          5 │            868 │
│          6 │            869 │
│          7 │            870 │
│          8 │            871 │
│          9 │            872 │
│         10 │            873 │
│          · │             ·  │
│          · │             ·  │
│          · │             ·  │
│       9991 │          10854 │
│       9992 │          10855 │
│       9993 │          10856 │
│       9994 │          10857 │
│       9995 │          10858 │
│       9996 │          10859 │
│       9997 │          10860 │
│       9998 │          10861 │
│       9999 │          10862 │
│      10000 │          10863 │
├────────────┴────────────────┤
│ ? rows            2 columns │
│ (>9999 rows, 20 shown)      │
└─────────────────────────────┘

In [10]:
ob_prod_certifications = duckdb.sql("""
select
    ob_prod_module_ids.product_ptr_id,
    coalesce(CertificationStandard_Value, '') as CertificationStandard_Value,
    csv.* exclude(CertificationStandard_Value, prodmodule_id),
from read_csv('ProdCertification.csv') as csv
join ob_prod_module_ids on ob_prod_module_ids.csv_row_id = csv.prodmodule_id
""").pl()
ob_prod_certifications

product_ptr_id,CertificationStandard_Value,CertificationDate_Value,csv_row_id
i64,str,date,i64
864,"""UL1703_2002""",null,3972
864,"""IEC61215_2016""",null,21543
864,"""IEC61853_1_2011""",null,42811
865,"""UL1703_2002""",null,17579
865,"""IEC61215_2016""",null,21544
…,…,…,…
21972,"""IEC61215_2016""",null,42651
21972,"""IEC61853_1_2011""",null,63919
21973,"""UL61730_2017""",null,20224


In [11]:
ob_prod_certification_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select * exclude(product_ptr_id) from ob_prod_certifications
    """),
    target_table_name='server_prodcertification',
    fk_fills=[
        ('Firmware_id', dict(
            target_table_name='server_firmware',
            pk='id',
            fk_fills=[
                ('Checksum_id', dict(target_table_name='server_checksum', pk='id')),
            ],
        )),
    ],
)
ob_prod_certification_ids

┌────────────────┬───────┐
│   csv_row_id   │  id   │
│     int64      │ int64 │
├────────────────┼───────┤
│              1 │  1215 │
│              2 │  1299 │
│              3 │  1302 │
│              4 │  1398 │
│              5 │  1407 │
│              6 │  1425 │
│              7 │  1500 │
│              8 │  1554 │
│              9 │  1683 │
│             10 │  1707 │
│              · │    ·  │
│              · │    ·  │
│              · │    ·  │
│          10058 │ 30515 │
│          10059 │ 30551 │
│          10060 │ 30602 │
│          10061 │ 30704 │
│          10062 │ 30806 │
│          10063 │ 30824 │
│          10064 │ 30842 │
│          10065 │ 30884 │
│          10066 │ 30992 │
│          10067 │ 31004 │
├────────────────┴───────┤
│ ? rows       2 columns │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [12]:
upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        ob_prod_certification_ids.csv_row_id,
        product_ptr_id as product_id,
        ob_prod_certification_ids.id as prodcertification_id,
    from ob_prod_certification_ids
    join ob_prod_certifications using (csv_row_id)
    join ob_prod_module_ids using (product_ptr_id)
    """),
    target_table_name='server_product_ProdCertifications',
)

┌────────────────┬───────┐
│   csv_row_id   │  id   │
│     int64      │ int64 │
├────────────────┼───────┤
│           3972 │   864 │
│          21543 │   865 │
│          42811 │   866 │
│          17579 │   867 │
│          21544 │   868 │
│          42812 │   869 │
│           3973 │   870 │
│          21545 │   871 │
│          42813 │   872 │
│           8061 │   873 │
│             ·  │    ·  │
│             ·  │    ·  │
│             ·  │    ·  │
│          24870 │ 10854 │
│          46138 │ 10855 │
│          15038 │ 10856 │
│          24871 │ 10857 │
│          46139 │ 10858 │
│           1559 │ 10859 │
│          24872 │ 10860 │
│          46140 │ 10861 │
│          17779 │ 10862 │
│          24873 │ 10863 │
├────────────────┴───────┤
│ ? rows       2 columns │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [13]:
ob_module_elect_ratings = pl.read_csv('ModuleElectRating.csv').with_columns(
    pl.col('^.+_Unit$').fill_null('')
)
ob_module_elect_ratings = duckdb.sql("""
select
    ob_prod_module_ids.product_ptr_id,
    csv.* exclude(prodmodule_id),
from ob_module_elect_ratings as csv
join ob_prod_module_ids on ob_prod_module_ids.csv_row_id = csv.prodmodule_id
""")
ob_module_elect_ratings.show(max_width=120)

┌────────────────┬──────────────────────┬──────────────┬───┬──────────────────────┬──────────────────────┬────────────┐
│ product_ptr_id │ ModuleRatingCondit…  │ PowerDC_Unit │ … │ VoltageAtMaximumPo…  │ VoltageAtMaximumPo…  │ csv_row_id │
│     int64      │       varchar        │   varchar    │   │       varchar        │        double        │   int64    │
├────────────────┼──────────────────────┼──────────────┼───┼──────────────────────┼──────────────────────┼────────────┤
│           1990 │ NOCT                 │              │ … │ V                    │                 31.7 │      64931 │
│           1991 │ STC                  │ W            │ … │ V                    │                 34.0 │       1128 │
│           1991 │ PTC                  │ W            │ … │                      │                 NULL │      22396 │
│           1991 │ LIC                  │              │ … │ V                    │                34.18 │      43664 │
│           1991 │ NOCT                 

In [14]:
ob_module_elect_rating_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select * exclude(product_ptr_id) from ob_module_elect_ratings
    """),
    target_table_name='server_moduleelectrating',
)
ob_module_elect_rating_ids

┌────────────────┬───────┐
│   csv_row_id   │  id   │
│     int64      │ int64 │
├────────────────┼───────┤
│              1 │     1 │
│          21269 │     2 │
│          42537 │     3 │
│          63805 │     4 │
│              2 │     5 │
│          21270 │     6 │
│          42538 │     7 │
│          63806 │     8 │
│              3 │     9 │
│          21271 │    10 │
│            ·   │     · │
│            ·   │     · │
│            ·   │     · │
│          45034 │  9991 │
│          66302 │  9992 │
│           2499 │  9993 │
│          23767 │  9994 │
│          45035 │  9995 │
│          66303 │  9996 │
│           2500 │  9997 │
│          23768 │  9998 │
│          45036 │  9999 │
│          66304 │ 10000 │
├────────────────┴───────┤
│ ? rows       2 columns │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [15]:
upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        ob_module_elect_rating_ids.csv_row_id,
        product_ptr_id as prodmodule_id,
        ob_module_elect_rating_ids.id as moduleelectrating_id,
    from ob_module_elect_rating_ids
    join ob_module_elect_ratings using (csv_row_id)
    join ob_prod_module_ids using (product_ptr_id)
    """),
    target_table_name='server_prodmodule_ModuleElectRatings',
)

┌────────────────┬───────┐
│   csv_row_id   │  id   │
│     int64      │ int64 │
├────────────────┼───────┤
│              1 │     1 │
│          21269 │     2 │
│          42537 │     3 │
│          63805 │     4 │
│              2 │     5 │
│          21270 │     6 │
│          42538 │     7 │
│          63806 │     8 │
│              3 │     9 │
│          21271 │    10 │
│            ·   │     · │
│            ·   │     · │
│            ·   │     · │
│          45034 │  9991 │
│          66302 │  9992 │
│           2499 │  9993 │
│          23767 │  9994 │
│          45035 │  9995 │
│          66303 │  9996 │
│           2500 │  9997 │
│          23768 │  9998 │
│          45036 │  9999 │
│          66304 │ 10000 │
├────────────────┴───────┤
│ ? rows       2 columns │
│ (>9999 rows, 20 shown) │
└────────────────────────┘

In [16]:
duckdb.sql("""
commit;
""")